In [1]:
import pandas as pd
import datetime 
import numpy as np
import pickle
import matplotlib.pyplot as plt

In [2]:
smileon = pd.read_csv('../data/smileon_edited.csv')
smileon.head()

C:\Users\Owner\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (12,17,59,62,78,84,90,94,106) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Unnamed: 0,patient_id,callreportnum,reportversion,callstart,callend,calllength,callstaff_id,entrystaff_id,entrydate,...,oce_9951,oce_0000_denturemandibular,oce_0000_denturemaxillary,oce_treatmentplan_developed,oce_treatmentplan_complete,oce_clientrestored,oce_WHATCAREGIVEN,tp_plancomplete,tp_plandeveloped,tp_clientrestored
0,0,3638169,93115324,registration,9/30/2021 16:56,9/30/2021 16:57,1,123747,123747,9/30/2021 16:57,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,4431194,93115188,registration,9/30/2021 16:53,9/30/2021 16:53,0,123747,123747,9/30/2021 16:53,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2941341,93115081,clinical,9/30/2021 16:50,9/30/2021 16:51,1,161979,161979,9/30/2021 16:51,...,NaN,NaN,NaN,NaN,NaN,NaN,274 - Bitewings - four films - 5; 1110 - Proph...,No,Yes,No
3,3,3823933,93115032,clinical,9/30/2021 16:48,9/30/2021 16:49,1,161979,161979,9/30/2021 16:49,...,NaN,NaN,NaN,NaN,NaN,NaN,2391 - Posterior one surface resin - 15; 2392...,No,Yes,No
4,4,3348256,93114968,clinical,9/30/2021 16:46,9/30/2021 16:48,2,161979,161979,9/30/2021 16:48,...,NaN,NaN,NaN,NaN,NaN,NaN,No Code - Denture step Mandibular: may be used...,No,Yes,No


In [3]:
smileon = smileon.sort_values(['patient_id', 'entrydate'])

In [4]:
smileon = smileon.reset_index()
smileon.head(70)

,index,Unnamed: 0,patient_id,callreportnum,reportversion,callstart,callend,calllength,callstaff_id,entrystaff_id,...,oce_9951,oce_0000_denturemandibular,oce_0000_denturemaxillary,oce_treatmentplan_developed,oce_treatmentplan_complete,oce_clientrestored,oce_WHATCAREGIVEN,tp_plancomplete,tp_plandeveloped,tp_clientrestored
0,13186,13186,-2,84708447,registration,2/19/2021 12:10,2/19/2021 12:56,46,119513,119513,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,47543,47543,-2,62945363,clinical,5/15/2019 10:00,5/15/2019 11:00,60,119473,119473,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN
2,42846,42846,-2,65438351,registration,8/13/2019 8:00,8/13/2019 8:05,5,133281,133281,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,228,228,-2,92968098,clinical,9/27/2021 12:56,9/27/2021 12:56,0,140162,140162,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,229,229,-2,92967776,registration,9/27/2021 12:51,9/27/2021 12:56,5,140162,140162,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,39680,39680,-1,66703310,clinical,9/27/2019 9:17,9/27/2019 9:18,1,118843,118843,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
66,39677,39677,-1,66703536,clinical,9/27/2019 9:26,9/27/2019 9:27,1,118843,118843,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67,39675,39675,-1,66703613,clinical,9/27/2019 9:28,9/27/2019 9:29,1,118843,118843,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
68,39673,39673,-1,66703692,clinical,9/27/2019 9:31,9/27/2019 9:32,1,118843,118843,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
#dropping the first 69 rows (rows 0-68) because they're weird - looks like test entries or something?  
#all data for patient's # -1 and -2
smileon = smileon.drop(range(0,69))
smileon = smileon.reset_index(drop=True)
smileon.head(10)

,index,Unnamed: 0,patient_id,callreportnum,reportversion,callstart,callend,calllength,callstaff_id,entrystaff_id,...,oce_9951,oce_0000_denturemandibular,oce_0000_denturemaxillary,oce_treatmentplan_developed,oce_treatmentplan_complete,oce_clientrestored,oce_WHATCAREGIVEN,tp_plancomplete,tp_plandeveloped,tp_clientrestored
0,32182,32182,2384106,70281146,clinical,1/20/2020 13:25,1/20/2020 13:25,0,134549,134549,...,NaN,NaN,NaN,NaN,NaN,NaN,No Code - Denture step Mandibular: may be used...,Yes,Yes,Yes
1,53004,53004,2384106,59821075,clinical,1/9/2019 10:07,1/9/2019 10:07,0,118821,118821,...,NaN,NaN,NaN,NaN,NaN,NaN,No Code - Denture step Mandibular: may be used...,NaN,Yes,NaN
2,34169,34169,2384106,69642939,clinical,12/16/2019 10:35,12/16/2019 10:35,0,134549,134549,...,NaN,NaN,NaN,NaN,NaN,NaN,No Code - Denture step Mandibular: may be used...,No,Yes,No
3,56013,56013,2384106,57055663,clinical,10/9/2018 18:59,10/9/2018 18:59,0,118821,118821,...,NaN,NaN,NaN,NaN,NaN,NaN,150 - Comprehensive Oral Evaluation -new or es...,NaN,NaN,NaN
4,38027,38027,2384106,67492062,clinical,10/21/2019 8:35,10/21/2019 8:35,0,134549,134549,...,NaN,NaN,NaN,NaN,NaN,NaN,5110 - Complete denture-maxillary - 60; 5120 -...,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,54010,54010,2384949,58809242,clinical,12/4/2018 14:09,12/4/2018 14:10,1,118821,118821,...,NaN,NaN,NaN,NaN,NaN,NaN,No Code - Denture step Mandibular: may be used...,Yes,Yes,Yes
66,14413,14413,2384949,83992293,clinical,1/26/2021 14:02,1/26/2021 14:04,2,134549,134549,...,NaN,NaN,NaN,NaN,NaN,NaN,180 - Comprehensive Periodontal Evaluation - 1...,Yes,Yes,Yes
67,51642,51642,2384949,60707724,clinical,2/18/2019 11:05,2/18/2019 11:05,0,118821,118821,...,NaN,NaN,NaN,NaN,NaN,NaN,1330 - Oral Hygiene Instructions - ; 1206 - ap...,NaN,NaN,NaN
68,28960,28960,2384949,71609435,clinical,3/2/2020 16:18,3/2/2020 16:19,1,134549,134549,...,NaN,NaN,NaN,NaN,NaN,NaN,No Code - Denture step Mandibular: may be used...,Yes,Yes,Yes


In [6]:
smileon['patient_id'].value_counts()

3212379    54
2441776    43
2758373    42
2431132    37
2416076    35
           ..
2628473     1
2530943     1
2511760     1
2635370     1
2823981     1
Name: patient_id, Length: 10709, dtype: int64

In [8]:
smileon.loc[smileon['patient_id']==2416076]

,index,Unnamed: 0,patient_id,callreportnum,reportversion,callstart,callend,calllength,callstaff_id,entrystaff_id,...,oce_9951,oce_0000_denturemandibular,oce_0000_denturemaxillary,oce_treatmentplan_developed,oce_treatmentplan_complete,oce_clientrestored,oce_WHATCAREGIVEN,tp_plancomplete,tp_plandeveloped,tp_clientrestored
2477,32756,32756,2416076,70080607,clinical,1/13/2020 12:49,1/13/2020 12:52,3,117268,117268,...,NaN,NaN,NaN,NaN,NaN,NaN,272 - Bitewings - two films - 5; 1110 - Prophy...,No,Yes,No
2478,33854,33854,2416076,69737230,clinical,12/19/2019 11:37,12/19/2019 11:38,1,116749,116749,...,NaN,NaN,NaN,NaN,NaN,NaN,2330 - Resin one surface - Anterior - 15,No,Yes,Yes
2479,55777,55777,2416076,57225628,clinical,10/17/2018 10:15,10/17/2018 10:16,1,118821,118821,...,NaN,NaN,NaN,NaN,NaN,NaN,5221 - Partial denture - resin base - immediat...,NaN,NaN,NaN
2480,56291,56291,2416076,56828901,clinical,10/2/2018 13:47,10/2/2018 13:49,2,118821,118821,...,NaN,NaN,NaN,NaN,NaN,NaN,150 - Comprehensive Oral Evaluation -new or es...,NaN,NaN,NaN
2481,56129,56129,2416076,56850717,clinical,10/5/2018 9:46,10/5/2018 9:55,9,118820,118820,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2482,18823,18823,2416076,80792354,clinical,10/26/2020 14:05,10/26/2020 14:08,3,134549,134549,...,NaN,NaN,NaN,NaN,NaN,NaN,220 - Intraoral - Periapical First Film - 5; 2...,No,Yes,No
2483,36240,36240,2416076,68190928,clinical,11/13/2019 10:17,11/13/2019 10:18,1,134549,134549,...,1.0,NaN,NaN,NaN,NaN,NaN,2330 - Resin one surface - Anterior - 15; 9951...,NaN,NaN,NaN
2484,54987,54987,2416076,57786508,clinical,11/7/2018 9:26,11/7/2018 9:27,1,118821,118821,...,NaN,NaN,NaN,NaN,NaN,NaN,No Code - Denture step Mandibular: may be used...,NaN,NaN,NaN
2485,54774,54774,2416076,58628791,clinical,11/12/2018 15:32,11/12/2018 15:33,1,118821,118821,...,NaN,NaN,NaN,NaN,NaN,NaN,No Code - Denture step Mandibular: may be used...,NaN,NaN,NaN
2486,54503,54503,2416076,58629436,clinical,11/19/2018 15:50,11/19/2018 15:50,0,118821,118821,...,NaN,NaN,NaN,NaN,NaN,NaN,No Code - Denture step Mandibular: may be used...,NaN,NaN,NaN


In [ ]:
smileon = smileon.sort_values(['patient_id', 'entrydate'])